In [1]:
import torch
import torch.nn.functional as F
from torch_geometric.loader import NeighborSampler
from utils import load_data, load_parameter, evaluate
from model import GraphAutoencoder, NoisePredictor, custom_readout

In [2]:
device, subset_size, verbose, sizes, batch_size, input_dim, hidden_dim, gnn_type = load_parameter()
data = load_data(subset_size, verbose, hidden_dim)

/home/zihend1/.conda/envs/py38/lib/python3.8/site-packages/anndata/_core/anndata.py:1840: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")
/home/zihend1/.conda/envs/py38/lib/python3.8/site-packages/anndata/_core/anndata.py:1840: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")
/home/zihend1/.conda/envs/py38/lib/python3.8/site-packages/scanpy/preprocessing/_normalization.py:169: UserWarning: Received a view of an AnnData. Making a copy.
  view_to_actual(adata)
/home/zihend1/.conda/envs/py38/lib/python3.8/site-packages/scanpy/preprocessing/_normalization.py:196: UserWarning: Some cells have zero counts
  warn(UserWarning('Some cells have zero counts'))


AnnData object details: AnnData object with n_obs × n_vars = 502009 × 3000
    var: 'gene_ids', 'feature_types', 'genome', 'n_cells', 'highly_variable', 'highly_variable_rank', 'means', 'variances', 'variances_norm'
    uns: 'hvg', 'log1p'
Index(['Xkr4', 'Sox17', 'Lypla1', 'Atp6v1h', 'Adhfe1', 'Lactb2', 'Sbspon',
       'Pi15', 'Tfap2b', 'Gsta3',
       ...
       'S100g', 'Grpr', 'Cltrn', 'Ace2', 'Vegfd', 'Piga', 'Egfl6', 'Tmsb4x',
       'Prps2', 'Zfy1'],
      dtype='object', length=3000)
Index(['s_008um_00301_00321-1', 's_008um_00602_00290-1',
       's_008um_00789_00234-1', 's_008um_00728_00006-1',
       's_008um_00526_00291-1', 's_008um_00681_00396-1',
       's_008um_00078_00444-1', 's_008um_00128_00278-1',
       's_008um_00052_00559-1', 's_008um_00121_00413-1',
       ...
       's_008um_00572_00609-1', 's_008um_00536_00521-1',
       's_008um_00821_00426-1', 's_008um_00808_00300-1',
       's_008um_00197_00639-1', 's_008um_00252_00421-1',
       's_008um_00653_00166-1', 's_0

Processing super nodes:  12%|█▏        | 30/256 [00:00<00:01, 115.80it/s]

Subgraph for super node 00000_00028 has 3 nodes and 2 edges.
Subgraph for super node 00000_00029 has 7 nodes and 6 edges.
Subgraph for super node 00000_00030 has 7 nodes and 6 edges.
Subgraph for super node 00000_00031 has 7 nodes and 6 edges.
Subgraph for super node 00000_00032 has 7 nodes and 6 edges.
Subgraph for super node 00000_00033 has 7 nodes and 6 edges.
Subgraph for super node 00000_00034 has 7 nodes and 6 edges.
Subgraph for super node 00000_00035 has 7 nodes and 6 edges.
Subgraph for super node 00000_00036 has 7 nodes and 6 edges.
Subgraph for super node 00000_00037 has 7 nodes and 6 edges.
Subgraph for super node 00000_00038 has 7 nodes and 6 edges.
Subgraph for super node 00000_00039 has 7 nodes and 6 edges.
Subgraph for super node 00000_00040 has 7 nodes and 6 edges.
Subgraph for super node 00000_00041 has 7 nodes and 6 edges.
Subgraph for super node 00000_00042 has 7 nodes and 6 edges.
Subgraph for super node 00000_00043 has 7 nodes and 6 edges.
Subgraph for super node 

Processing super nodes:  23%|██▎       | 59/256 [00:00<00:01, 177.18it/s]

...

Subgraph for super node 00003_00063 has 49 nodes and 156 edges.
Subgraph for super node 00003_00064 has 49 nodes and 156 edges.
Subgraph for super node 00003_00065 has 49 nodes and 156 edges.
Subgraph for super node 00003_00066 has 49 nodes and 156 edges.


Processing super nodes:  99%|█████████▉| 254/256 [00:12<00:00, 15.10it/s]

Subgraph for super node 00003_00067 has 49 nodes and 156 edges.
Subgraph for super node 00003_00068 has 49 nodes and 156 edges.
Subgraph for super node 00003_00069 has 49 nodes and 156 edges.
Subgraph for super node 00003_00070 has 49 nodes and 156 edges.


Processing super nodes: 100%|██████████| 256/256 [00:13<00:00, 19.65it/s]


Subgraph for super node 00003_00071 has 49 nodes and 156 edges.


In [3]:
sampler = NeighborSampler(edge_index=data.super_node_edge, sizes=sizes, batch_size=batch_size, shuffle=True, num_workers=0)
model = GraphAutoencoder(input_dim=input_dim, hidden_dim=hidden_dim, gnn_type=gnn_type).to(device)
noise_predictor = NoisePredictor(hidden_dim=hidden_dim, output_dim=input_dim).to(device)
optimizer = torch.optim.Adam(list(model.parameters()) + list(noise_predictor.parameters()) + list(data.normal_node_hidden.parameters()), lr=0.01, weight_decay=0.0001)
criterion = torch.nn.MSELoss()

In [4]:
epochs = 10
for epoch in range(epochs):
    model.train()
    noise_predictor.train()
    total_loss = 0
    for batch_size, n_id, adjs in sampler:
        optimizer.zero_grad()
        x = data.super_node_features[n_id].to(device)  # Node feature matrix

        adjs = [adj.to(device) for adj in adjs]
        edge_indices = [adj.edge_index for adj in adjs]

        z, logits = model(x, edge_indices)
        
        # Compute autoencoder loss
        loss = criterion(logits, x)

        # Compute loss for normal node hidden representations
        for i, super_node_id in enumerate(n_id):
            super_node = data.inverse_super_node_mapping[super_node_id.item()]
            subgraph = data.subgraphs[super_node]

            normal_node_hidden = torch.stack([data.normal_node_hidden[node] for node in subgraph.nodes]).to(device)
            super_node_hidden = z[i].to(device)  # Use local batch index i

            readout_result = custom_readout(normal_node_hidden)
            predicted_noise = noise_predictor(super_node_hidden)

            # Get edge indices for normal nodes
            normal_edge_index = data.get_normal_node_edge_index(super_node).to(device)

            # Decode normal node hidden representations to gene expression
            node_gene_expression = F.relu(model.decoder_layer2(model.decoder_layer1(normal_node_hidden, normal_edge_index), normal_edge_index))

            # Compute the reverse log1p transformation
            real_node_gene_expression = torch.exp(node_gene_expression) - 1
            real_predicted_noise = torch.exp(predicted_noise) - 1

            # Sum real node gene expressions and add the real predicted noise
            combined_gene_expression = real_node_gene_expression.sum(dim=0) + real_predicted_noise
            
            # Convert back to log1p for computing MSE loss
            combined_gene_expression_log1p = torch.log1p(combined_gene_expression)

            # Compute the reverse log1p transformation for super node gene expression
            super_node_gene_expression = torch.tensor(data.super_node_data.loc[super_node].values, dtype=torch.float).to(device)

            # Compute gene expression loss
            gene_expression_loss = criterion(combined_gene_expression_log1p, super_node_gene_expression)

            # Minimize readout loss and gene expression loss
            loss += criterion(readout_result, super_node_hidden) + gene_expression_loss

        total_loss += loss.item()
        loss.backward()
        optimizer.step()

    print(f'Epoch {epoch+1}, Loss: {total_loss/len(sampler)}')

Epoch 1, Loss: 916.2889003753662
Epoch 2, Loss: 137.91349506378174
Epoch 3, Loss: 122.36096000671387
Epoch 4, Loss: 106.85577201843262
Epoch 5, Loss: 93.84435749053955
Epoch 6, Loss: 91.88889598846436
Epoch 7, Loss: 79.97829341888428
Epoch 8, Loss: 75.13064193725586
Epoch 9, Loss: 68.83605527877808
Epoch 10, Loss: 64.89547967910767


In [5]:
mse_loss = evaluate(model, data, device)

In [6]:
print(f'Average MSE Loss: {mse_loss}')

Average MSE Loss: 0.6101446039634372


In [16]:
model.eval()
criterion = torch.nn.MSELoss()
total_mse_loss = 0
total_nodes = 0

sampler = NeighborSampler(edge_index=data.super_node_edge, sizes=[-1]*2, batch_size=batch_size, shuffle=False, num_workers=0)

with torch.no_grad():
    for batch_size, n_id, adjs in sampler:
        for i, super_node_id in enumerate(n_id):
            super_node = data.inverse_super_node_mapping[super_node_id.item()]
            subgraph = data.subgraphs[super_node]
            normal_node_hidden = torch.stack([data.normal_node_hidden[node] for node in subgraph.nodes]).to(device)

            # Get edge indices for normal nodes
            normal_edge_index = data.get_normal_node_edge_index(super_node).to(device)

            # Decode normal node hidden representations to gene expression
            node_gene_expression = F.relu(model.decoder_layer2(model.decoder_layer1(normal_node_hidden, normal_edge_index), normal_edge_index))

            # Get the real node gene expressions from the dataset
            real_node_gene_expression = torch.stack([
                torch.tensor(subgraph.nodes[node]['features'].todense(), dtype=torch.float).to(device) 
                if hasattr(subgraph.nodes[node]['features'], 'todense') 
                else torch.tensor(subgraph.nodes[node]['features'], dtype=torch.float).to(device)
                for node in subgraph.nodes
            ])

            # Ensure the real_node_gene_expression has the same shape as node_gene_expression
            real_node_gene_expression = real_node_gene_expression.squeeze()

            # Compute the MSE loss for this subgraph
            mse_loss = criterion(node_gene_expression, real_node_gene_expression)
            total_mse_loss += mse_loss.item() * len(subgraph.nodes)
            total_nodes += len(subgraph.nodes)
            
            break
        break

In [17]:
node_gene_expression

tensor([[0.0000, 0.0000, 0.3093,  ..., 1.7027, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.3387,  ..., 1.2476, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.3719,  ..., 1.7987, 0.0000, 0.0000]],
       device='cuda:0')

In [18]:
real_node_gene_expression

tensor([[0.0000, 0.0000, 0.0000,  ..., 4.9092, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000,  ..., 5.9050, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000,  ..., 4.5118, 0.0000, 0.0000]],
       device='cuda:0')

In [19]:
for node in subgraph.nodes:
    print(subgraph.nodes[node]['features'].todense())

[[0.       0.       0.       ... 4.909187 0.       0.      ]]
[[0.       0.       0.       ... 5.904963 0.       0.      ]]
[[0.       0.       0.       ... 4.511849 0.       0.      ]]


In [20]:
data.super_node_features[data.super_node_mapping[super_node]]

tensor([0.0000, 0.0000, 0.0000,  ..., 5.1315, 0.0000, 0.0000])